In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
saved_model_dir = '/content/drive/MyDrive/LeafImageGeneration'


In [5]:
from torch import autocast
import torch
import matplotlib.pyplot as plt
import os
import random
from diffusers import UNet2DModel, DDPMScheduler, DDPMPipeline

# Function to generate high-quality images in batches of 10 from the saved model
def generate_images_from_saved_model(saved_model_dir, config):
    # Load the trained model from the saved directory
    model = UNet2DModel.from_pretrained(saved_model_dir).to("cuda")  # Move to GPU

    # Set up the noise scheduler (use same number of timesteps as during training)
    noise_scheduler = DDPMScheduler(num_train_timesteps=500)

    # Set up the pipeline for image generation
    pipeline = DDPMPipeline(unet=model, scheduler=noise_scheduler)

    # Set inference steps to 500 (must be <= training timesteps)
    num_inference_steps = 500  # Set to 500 to match the model's training configuration
    batch_size = 10  # Generate 10 images at a time

    # Generate a new random seed for each batch of images to ensure diversity
    random_seed = random.randint(0, 10000)  # Random seed for each generation
    print(f"Using random seed: {random_seed}")

    # Use full precision (float32) for maximum quality
    with torch.no_grad():
        model.eval()  # Set model to evaluation mode
        images = pipeline(batch_size=batch_size, generator=torch.Generator().manual_seed(random_seed), num_inference_steps=num_inference_steps).images

    # Save the generated images to the output directory
    os.makedirs(config.output_dir, exist_ok=True)

    # Create a grid of images (Batch of 10)
    fig, axes = plt.subplots(2, 5, figsize=(10, 4))
    for ax, img in zip(axes.flatten(), images):
        ax.imshow(img)
        ax.axis("off")

    plt.tight_layout()
    image_path = os.path.join(config.output_dir, f"generated_from_saved_model_batch_10_high_quality_{random_seed}.png")
    plt.savefig(image_path)
    plt.close()
    print(f"🖼️ Generated diverse images saved at {image_path}")

# Define config object
class Config:
    output_dir = "/content/drive/MyDrive/LeafImageGeneration/results"

config = Config()

# Path to the saved model
saved_model_dir = '/content/drive/MyDrive/LeafImageGeneration'  # Update with your model path

# Generate images using the saved model
generate_images_from_saved_model(saved_model_dir, config)


Using random seed: 6044


  0%|          | 0/500 [00:00<?, ?it/s]

🖼️ Generated diverse images saved at /content/drive/MyDrive/LeafImageGeneration/results/generated_from_saved_model_batch_10_high_quality_6044.png
